# Manual clustering with phy

In [1]:
from phy.cluster.manual.session import Session

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%gui qt

In [3]:
filename = "/data/spikesorting/4_20141130/20141130_all.kwik"

Recover the backup if it exists.

In [5]:
import os.path as op
import shutil
bak = filename + '.bak'
if op.exists(bak):
    shutil.copyfile(bak, filename)

Open the file.

In [6]:
session = Session()

In [7]:
session.open(filename)

In [8]:
model = session.model

In [9]:
clustering = session.clustering

In [12]:
wizard = session.wizard

In [10]:
model.filename

'/data/spikesorting/4_20141130/20141130_all.kwik'

## Using a custom quality measure to find best clusters

The function `session.best_clusters()` is a quick convenience function that accepts any `cluster => quality` function. It returns the list of best clusters by decreasing of quality, according to that function.

In [13]:
session.best_clusters(session.cluster_store.n_unmasked_channels, 10)

[1, 654, 598, 139, 556, 499, 136, 273, 572, 52]

> Note: Using `n_unmasked_channels` as a quality criterion is not a particularly good idea.

In most cases, it is probably a better idea to use the wizard (see below).

## Wizard

Let's open some views.

In [32]:
session.show_waveforms()
session.show_features()
session.show_correlograms()

Let's have a look at the best clusters. By default, the quality measure is the maximum mean_mask over all channels. You can use a different metrics with `wizard.set_quality(func)` where `func(cluster)` returns the quality. You can access the features and masks with `session.cluster_store.features(clusters)` and similar for masks.

In [18]:
clusters = wizard.best_clusters()

In [19]:
clusters

[1, 4, 6, 11, 12, 14, 15, 19, 20, 25]

Let's select one supposedly good cluster.

In [33]:
session.select([4])

What are the most similar clusters?

In [34]:
wizard.most_similar_clusters(4)

[254, 354, 291, 711, 474, 381, 587, 446, 123, 370]

> Note: if you can omit the argument to that function, it will default to the best cluster.

Let's select the most similar cluster along with cluster 4.

In [40]:
clusters = [4, 254]

In [41]:
session.select(clusters)

Let's merge these clusters.

In [42]:
session.merge(clusters)

> Note: You can undo/redo with `session.undo()` and `session.redo()`.

Let's check that the merge worked:

In [43]:
clustering.spikes_in_clusters(clusters)

array([], dtype=int64)

There are no spikes in the merged clusters: they are now in the newly-created cluster.

What is the id of the newly-created cluster?

In [44]:
session.selector.selected_clusters

array([720])

> Note: the `session.selector` object is responsible for the current selection of clusters and spikes.

In [45]:
clustering.spikes_in_clusters([720])

array([     44,    1324,    2399, ..., 3387687, 3387717, 3387916])

If we undo, we'll see that the spikes belonging to the two clusters we merged are the same:

In [46]:
session.undo()

In [47]:
clustering.spikes_in_clusters(clusters)

array([     44,    1324,    2399, ..., 3387687, 3387717, 3387916])

## Moving clusters to groups

Let's move some clusters. First, what is the group of the new cluster?

In [48]:
session.cluster_metadata.group(720)

3

Here are the default groups:

0. Noise
1. MUA
2. Good
3. Unsorted

> Note: adding new cluster groups is still a WIP feature.

Let's move a cluster to MUA:

In [49]:
session.move([720], 2)

In [50]:
session.cluster_metadata.group(720)

2

You can also undo/redo cluster groups:

In [51]:
session.undo()

In [52]:
session.cluster_metadata.group(720)

3

In [53]:
session.redo()

In [54]:
session.cluster_metadata.group(720)

2

## Split

Let's do a split. We're going to take half of the spikes from our two clusters and put them in a new cluster:

In [55]:
spikes = clustering.spikes_in_clusters(clusters)
spikes

array([     44,    1324,    2399, ..., 3387687, 3387717, 3387916])

In [56]:
session.split(spikes[::2])

## Save

Save to kwik.

In [59]:
session.save()

2015-04-13 21:13:40  session:452             Saved /data/spikesorting/4_20141130/20141130_all.kwik.
